In [ ]:
import sys
sys.path.insert(0, "../../")

import numpy as np
import matplotlib.pyplot as plt
from mixture.model import MixtureLognormalModel
from mixture.binomial_pricer import price_american_option

Ks = np.linspace(80, 120, 10)
Ts = np.array([0.5, 1.0, 1.5, 2.0, 2.5])
S0 = 100
r = np.full_like(Ts, 0.02)
q = np.full_like(Ts, 0.01)
b = np.full_like(Ts, 0.015)
sigma = 0.25
is_call = True
cash_divs = np.array([[0.3, 0.5], [1.0, 0.6]])

# Generate de-Americanized prices
european_prices = []
for T in Ts:
    row = []
    for K in Ks:
        euro_price, premium = price_american_option(S0, K, T, 200, r[0], q[0], b[0], sigma, is_call, cash_divs)
        row.append(euro_price)
    european_prices.append(row)
european_prices = np.array(european_prices)

# Fit the mixture model
model = MixtureLognormalModel(n_components=2)
model.fit_to_prices(european_prices, Ks, Ts, S0, r, q, b, is_call, cash_divs)

fitted_prices, _ = model.price_and_greeks(S0, Ks, Ts, r, q, b, is_call, cash_divs)

# Plot results
plt.figure(figsize=(10, 5))
for i, T in enumerate(Ts):
    plt.plot(Ks, european_prices[i], "o", label=f"De-Americanized T={T}")
    plt.plot(Ks, fitted_prices[i], "--", label=f"Fitted T={T}")
plt.title("Fitting Mixture Model to De-Americanized Prices")
plt.xlabel("Strike")
plt.ylabel("Option Price")
plt.grid(True)
plt.legend()
plt.show()
